In [2]:
from training.model import Model
import pandas as pd

In [4]:
model = Model("nlptown/bert-base-multilingual-uncased-sentiment")

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at nlptown/bert-base-multilingual-uncased-sentiment.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [5]:
data_json = pd.read_json("data/preprocessed/covid", orient="records", lines=True)
dataset = Model.create_dataset(data_json)

In [6]:
tf_dataset = model.prepare_dataset(dataset)

c:\users\yaroslav\appdata\local\programs\python\python38\lib\site-packages\transformers\generation\tf_utils.py:465: UserWarning: `seed_generator` is deprecated and will be removed in a future version.
  warnings.warn("`seed_generator` is deprecated and will be removed in a future version.", UserWarning)


Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...bert
......vars
...bert\embeddings
......vars
.........0
.........1
.........2
...bert\embeddings\LayerNorm
......vars
.........0
.........1
...bert\embeddings\dropout
......vars
...bert\encoder
......vars
...bert\encoder\layer\tf_bert_layer
......vars
...bert\encoder\layer\tf_bert_layer\attention
......vars
...bert\encoder\layer\tf_bert_layer\attention\dense_output
......vars
...bert\encoder\layer\tf_bert_layer\attention\dense_output\LayerNorm
......vars
.........0
.........1
...bert\encoder\layer\tf_bert_layer\attention\dense_output\dense
......vars
.........0
.........1
...bert\encoder\layer\tf_bert_layer\attention\dense_output\dropout
......vars
...bert\encoder\layer\tf_bert_layer\attention\self_attention
......vars
...bert\encoder\layer\tf_bert_layer\attention\self_attention\dropout
......vars
...bert\encoder\layer\tf_bert_layer\attention\self_attention\key
......vars
.........0
.........1
...bert\encoder\layer\t

Map:   0%|          | 0/10201 [00:00<?, ? examples/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [7]:
model.fit(tf_dataset)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


637/637 [==============================] - 1909s 3s/step - loss: 0.1308


In [8]:
model.save_model("model/covid.h5py")

In [12]:
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import tensorflow as tf

loaded_model = TFAutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
loaded_model.load_weights('model/covid/tf_model.h5')
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

Some layers from the model checkpoint at nlptown/bert-base-multilingual-uncased-sentiment were not used when initializing TFBertForSequenceClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at nlptown/bert-base-multilingual-uncased-sentiment.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [17]:
input_text = "paid rent march april give money back"
input_text_tokenized = tokenizer.encode(input_text,
                                            truncation=True,
                                            padding=True,
                                            return_tensors="tf")
prediction = loaded_model(input_text_tokenized)
prediction_logits = prediction[0]
prediction_probs = tf.nn.softmax(prediction_logits,axis=1).numpy()
print(f'The prediction probs are: {prediction_probs}')

The prediction probs are: [[9.8647124e-01 1.3396989e-02 5.6076202e-05 3.5006189e-05 4.0732859e-05]]


In [18]:
logits = loaded_model(input_text_tokenized).logits
predicted_class_id = int(tf.math.argmax(logits, axis=-1)[0])
loaded_model.config.id2label[predicted_class_id]

'1 star'

In [20]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis", model="model/covid")
classifier("president donald trump announce scientist finally found vaccine cure corona virus")

Some layers from the model checkpoint at model/covid were not used when initializing TFBertForSequenceClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at model/covid.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


[{'label': '1 star', 'score': 0.9956220984458923}]

In [ ]:
model.model.predict()